# Tesseract

Der hier besprochene Code soll ein kleines Beispiel dafür sein, was im Umgang mit Tesseract und den vom neuronalen Netz erkannten Bounding Boxes für die Kategorie Text möglich ist und wo Limitierungen bestehen.

In [ ]:
from PIL import Image
from os.path import isfile, isdir

import pytesseract, json, os, cv2
import numpy as np

#### Funktion  
Für die Erfassung der Image-Dateien wurde die folgende Funktion geschrieben. Ihr wird der Dateipfad und ein leeres Set übergeben.  
Die Funktion iteriert über jedes Element im Hauptordner. Stößt es auf ein Element, das wiederum ein Ordner ist, ruft sie sich selbst auf - sie iteriert solange durch jeden Ordner bis sie das letzte Element des Hauptordners abgearbeitet hat.  
Die Funktion gibt eine geordnete Liste zurück.

In [ ]:
def get_images(base_path, all_image_names):
    file_name_list = os.listdir(base_path)
    
    for element in file_name_list:
        if isfile(base_path + element) & (element.lower().endswith(".jpg")):
            all_image_names.add((base_path + element).replace(DATASET_PATH, ''))
        elif isdir(base_path + element):
            get_images(base_path + element + "/", all_image_names)

    all_image_names = list(all_image_names)
    all_image_names.sort()
    return all_image_names

In [ ]:
WORK_DIR = "/home/julius/PowerFolders/Masterarbeit/"
os.chdir(WORK_DIR)

DATASET_PATH = "./1_Datensaetze/data100/"
JSON_PATH = "./detections/data100/16,04,2021-12,43/"

Benötigt wird neben den Bildern die '.json' Datei in der die Koordinaten der Bounding Boxes gespeichert worden sind

In [ ]:
images = set()
images = get_images(DATASET_PATH, images)

with open(JSON_PATH + "bounding_boxes.json", "r+") as inputfile:
    bounding_boxes = json.load(inputfile)

In einem ersten Schritt werden die Bounding Boxes ermittelt, die zum Label "Text" gehören. Sie werden  für jedes Bild in eine Liste gespeichert. Sollte kein Textfeld erkannt worden sein, wird eine leere Liste angelegt.

In [ ]:
boxes_per_image = {}
for count in range(len(images)):
    current_image = bounding_boxes[images[count]]

    for count_two in range(len(current_image["category_names"])):
        if current_image["category_names"][count_two] == "text":
            if images[count] not in boxes_per_image:
                boxes_per_image[images[count]] = [current_image["prediction_boxes"][count_two]]
            else:
                boxes_per_image[images[count]].append(current_image["prediction_boxes"][count_two])

    # if theres no text box in image, create an empty list
    if images[count] not in boxes_per_image:
        boxes_per_image[images[count]] = []

#### Funktion

In der Funktion _pic_split_ werden die Farbkanäle eines gegebenen Bildes aufgeteilt und zusätzlich binarisiert. Dies wird gemacht um einen weiteren Informationsgewinn aus den Bildern zu ziehen. Neben den vollfarbigen Bildern und den aus diesen generierten graustufenbildern kommen drei weitere Varianten hinzu, die das Abgebildete in anderen Kontrasten zeigen.

In [ ]:
def pic_split(img):

    #converting into numpy
    img_array = np.array(img).astype(float)

    #the actual split
    img_array_blue = img_array[:,:,0]
    img_array_green = img_array[:,:,1]
    img_array_red = img_array[:,:,2]

    np.where(img_array_blue<128, 1, 0)
    np.where(img_array_green<128, 1, 0)
    np.where(img_array_red<128,1 ,0)

    #converting them back
    img_blue = Image.fromarray(img_array_blue.astype(np.uint8))
    img_green = Image.fromarray(img_array_green.astype(np.uint8))
    img_red = Image.fromarray(img_array_red.astype(np.uint8))

    return img_blue, img_green, img_red

Die folgende for-Schleife bildet die hauptsächliche Analyse der Bounding Boxes. Hierfür werden zunächst aus den Bildern die Bereiche ausgeschnitten, die in den Boxen liegen. Der Bildinhalt jeder Box wird anschließend transformiert. Neben dem vollfarbigen Bild wird ein Graustufenbild generiert. Um auch die Werte aus den einzelnen Farbschichten zu analysieren, werden mit der _pic_split_ Funktion drei weitere Bilder erzeugt. Die fünf Bilder zeigen jeweils den gleichen Bildinhalt, jedoch unterscheiden sich ihre jeweiligen Pixelwerte.  
Dies wird für die Anwendung von OCR wichtig, da bessere Ergebnisse meist mit einem starken Kontrast zwischen den Buchstaben und dem Hintergrund zusammenhängen. In jeder dieser Versionen ist der Kontrast zwischen den einzelnen Pixeln unterschiedlich groß, was zu unterschiedlichen Ergebnissen bei gleichem Bildinhalt führt.  
Was sich zeitlich leider nicht mehr realisieren ließ, ist eine weitere Analyse der einzelnen Pixelmatrizen. Es ist denkbar, dass ein Mittelwert der Matrizen bereits Rückschlüsse gibt, ob sich eine bestimmte Variante besser oder weniger gut für OCR eignet. So könnte im Vorfeld am Mittelwert ermittelt werden, welche Varianten am ehesten brauchbare Ergebnisse liefern und nur diese weiter verwenden. Denn mit jeder Bildvariante wird auch der erzeugte Output größer. Mit diesem muss wiederum umgegangen werden, was mehrere Fallstricke beinhaltet.  
In diesem Fall wurde die Zeilenstruktur der erkannten Strings beibehalten und nur nach einem von tesseract erkannten Zeilenumbruch getrennt. Dies führt dazu, dass mehrere Worte in einen String fallen, wodurch der Kontext erhalten bleibt. Eine andere Möglichkeit wäre die Trennung nach Leerzeichen gewesen. Durch eine teilweise recht willkürlich wirkende Setzung von Leerzeichen, kann es jedoch sein, dass diese angesprochenen Kontexte verloren gehen. So erkennt tesseract in manchen Datumsangaben Leerzeichen zwischen den Zahlen und Worten. Auch wenn diese nicht immer deplaziert sein müssen würde eine Datumsangabe wie "13. November 1984" durch eine Leerzeichentrennung auseinanderfallen. Tauchen mehrer Daten auf, können diese einzelnen Elemente dann vermischt werden und der Sinn verloren gehen. Das gleiche gilt für Wortpaare.  
Eine weitere Schwierigkeit ist der Umgang mit Dopplungen. Durch die Anwendung von OCR auf fünf Varianten des gleichen Inhalts können auch fünf Strings mit gleichem oder ähnlichem Inhalt entstehen. Reine Dubletten können über einen Vergleich der von Leerzeichen bereinigten Strings aussortiert werden. Strings, die sich nur in einzelnen Buchstaben von einander unterscheiden, würden beibehalten werden.  
Diese müssten zusammen mit den anderen Strings ohne bedeutsamen Inhalt in einem weiteren Schritt ebenfalls gefiltert werden. Denkbare Szenarien wären der Abgleich mit Wörterbüchern und der Ausschluss von einzelnen Worten mit einer Wortlänge von unter eins oder zwei Buchstaben.  
Was ebenfalls erheblich zur Verbesserung der Ergebnisse führen kann ist eine Erkennung der Sprache bereits auf der Ebene der Objekterkennung. So wäre es möglich tesseract explizit zu sagen, welche Sprache zu erkennen ist, was die Erkennung von Sonderzeichen deutlich verbessern sollte.  
Dies alles macht deutlich, dass es Vorteilhaft wäre, neben dem reinen "Text" Label spezifischere Label wie "Datum" und "Ortsangabe" beziehungsweise ein Sprachlabel zu haben. Somit könnten unterschiedliche Verfahrensweisen mit dem zu erkennenden Text etabliert und die gesamte Qualität der Ergebnisse gesteigert werden.

In [ ]:
for element in range(len(images)):
    # load image
    image_dict = {}
    image_dict["complete_image"] = cv2.imread(DATASET_PATH + images[element])
    dump_dict = {}

    ###CONSOLE OUTPUT###
    print("[INFO] Read Image: {}.".format(images[element]))

    # go over every box in the image
    for box in range(len(boxes_per_image[images[element]])):
        # get box cords, length and with and cut text out
        x_one = boxes_per_image[images[element]][box][0]
        x_two = boxes_per_image[images[element]][box][1]
        y_one = boxes_per_image[images[element]][box][2]
        y_two = boxes_per_image[images[element]][box][3]

        box_width = abs(x_one - y_one)
        box_height = abs(x_two - y_two)

        image_dict["colored_box"] = image_dict["complete_image"][x_two : x_two + box_height, x_one : x_one + box_width]
        image_dict["blue_box"], image_dict["green_box"], image_dict["red_box"] = pic_split(image_dict["colored_box"])
        image_dict["gray_box"] = cv2.cvtColor(image_dict["colored_box"], cv2.COLOR_BGR2GRAY)

        # OCR on every variant
        for variant in image_dict:
            text_dump = set()

            if (variant != "complete_image"):
                image_text = "{}".format(pytesseract.image_to_string(image_dict[variant], lang="deu"))
                image_text = image_text.replace("\x0c", "").split("\n")
            else:
                continue

            if len(image_text) > 0:
                [text_dump.add(text) for text in image_text if len(text) > 0]
            
            if variant not in dump_dict:
                dump_dict[variant] = list(text_dump)
            else:
                dump_dict[variant].append(list(text_dump))
            
        ###CONSOLE OUTPUT###
        print("[INFO] Finished Detection for box {}/{} of image {}".format(box+1, len(boxes_per_image[images[element]]), images[element]))

    # OCR on the whole image
    image_text = "{}".format(pytesseract.image_to_string(image_dict["complete_image"], lang="deu"))
    image_text = image_text.replace("\x0c", "").split("\n")
    if len(image_text) > 0:
        [text_dump.add(text) for text in image_text if len(text) > 0]
    dump_dict["complete_image"] = list(text_dump)

    # dump results
    with open(JSON_PATH + "{}.json".format(images[element].split("/")[-1][:-4]), "w+", encoding="utf8") as output_file:
        json.dump(dump_dict, output_file, indent=4, ensure_ascii=False)